In [4]:
import copy
import itertools
import pickle
import warnings

import pandas as pd
import numpy as np

In [5]:
# Load dictionaries corresponding to ASHE tables [ONS]
with open('../data/processed/ASHE_dicts_all.pickle', 'rb') as input_file:
    ASHE_dicts_all = pickle.load(input_file)
    
# Load MS job dictionaries
with open('../data/processed/MSW_all.pickle', 'rb') as input_file:
    MSW_all = pickle.load(input_file)

In [6]:
# Hourly or annual salary data
HRAN = 'annual'

# Full-time or part-time workers
FTPT = 'FT'

# Weight type
WEIGHT = 'C'

# Specify which MSJ catalogue to use for salary calculations
MSW = MSW_all[f'{WEIGHT}_{HRAN}_{FTPT}']

In [7]:
hr_to_annual = 1 if HRAN == 'annual' else 37*52 #mean paid hrs worked

## 2011 salaries

In [8]:
ASHE_dict = ASHE_dicts_all[f'2011r_{HRAN}_{FTPT}']

In [9]:
# Difference with and without correction of order £100!
SE_correct = False

tot_pounds, tot_jobs = 0, 0

# NB Deloitte used 2011 salary data
for code, n_jobs in zip(MSW['MSW_2011_SOC2010']['SOC'].values,
                             MSW['MSW_2011_SOC2010']['MS_jobs'].values):
    mean_salary = ASHE_dict['mean_salary'][ASHE_dict['SOC2010_code']==code]
    if len(mean_salary)==1 and (not np.isnan(mean_salary[0])):

        if SE_correct:
            n_jobs *= (1-get_SE_frac_from_SOC2010(code,'2011'))
        tot_pounds += mean_salary[0]*n_jobs
        tot_jobs+= n_jobs
        
MS_mean = tot_pounds/tot_jobs*hr_to_annual

# UK overall mean salary
UK_mean = (np.nansum(ASHE_dict['mean_salary']*ASHE_dict['no_jobs'])/
      np.nansum(ASHE_dict['no_jobs'])*hr_to_annual)

In [10]:
print('=== 2011 salaries ===\n')
print(f'MS mean: £{MS_mean:,.0f}; UK mean: £{UK_mean:,.0f}')
# Ratios for means and medians
print(f'MS premium (mean): {100*(MS_mean/UK_mean-1):.0f}%')

=== 2011 salaries ===

MS mean: £42,758; UK mean: £32,663
MS premium (mean): 31%


## 2023 salaries

In [11]:
ASHE_dict = ASHE_dicts_all[f'2023p_{HRAN}_{FTPT}']
MSW = MSW_all[f'{WEIGHT}_{HRAN}_{FTPT}']

In [12]:
ASHE_dict = ASHE_dicts_all[f'2023p_{HRAN}_{FTPT}']
MSW = MSW_all[f'{WEIGHT}_{HRAN}_{FTPT}']

soc_codes, ms_jobs =  (MSW['MSW_2023_SOC2020']['SOC'].values, 
                       MSW['MSW_2023_SOC2020']['MS_jobs'].values)

### Synthetise salary distributions for 2023 based on ASHE percentiles

In [13]:
# All percentile levels in ASHE tables
pctl_levels_all = ['10', '20', '30', '40', '50', '60', '70', '80', '90']

N_samples = 10 # how many synthetic salaries to add for each percentile
valid_dist = np.zeros(4)
ms_salary_synth = [] # store percentile matched salary samples across all occupations

for i, (code, lvls, salaries) in enumerate(zip(
    ASHE_dict['SOC2020_code'], ASHE_dict['pctl_lvls'], ASHE_dict['pctl_salaries'])):
    curr_synth_dist, multipliers = [], []

    if set(lvls) == set(pctl_levels_all): # if all (10-90th) pctls available
        valid_dist[0]+=1
        curr_pctl_levels = np.copy(pctl_levels_all[0::1])
        curr_salary_levels = salaries[0::1]
        multiplier = 1
    elif set(pctl_levels_all[1:-1]).issubset(set(lvls)): # if 20-80th pctls only
        valid_dist[1]+=1
        curr_pctl_levels = pctl_levels_all[1:-1:1]
        curr_salary_levels = salaries[1:-1:1]
        multiplier = 2
    elif set(pctl_levels_all[2:-2]).issubset(set(lvls)): # if 30-70th pctls only
        valid_dist[2]+=1
        curr_pctl_levels = pctl_levels_all[2:-2:1]
        curr_salary_levels = salaries[2:-2:1]
        multiplier = 3
    elif set(pctl_levels_all[3:-3]).issubset(set(lvls)): # if 40-60th pctls only
        valid_dist[3]+=1
        curr_pctl_levels = pctl_levels_all[3:-3:1]
        curr_salary_levels = salaries[3:-3:1]
        multiplier = 4
    else:
        curr_pctl_levels = []
        curr_salary_levels = []
        multiplier = np.nan

    if not np.isnan(multiplier):
        multipliers = np.ones(len(curr_pctl_levels))
        multipliers[0] = multiplier
        multipliers[-1] = multiplier
    
        
    # --- diagnostics ---
    # print(curr_salary_levels)
    # print(multipliers)
    #print(len(curr_salary_levels)-len(multipliers)) # should always be 0
    # print(np.sum(multipliers), '|', end='') # should all be 9 or 0
    
    for j, (salary, mpl) in enumerate(zip(curr_salary_levels, multipliers)):
        curr_synth_dist.extend([salary]*int(N_samples*mpl))
    
    curr_synth_dist = sorted(curr_synth_dist)
    
    # Optional: adjust synth dist mean to match ONS tabulated mean
    ADJUST_MEANS = True
    # Can modify <5th/>95th percentiles without affected
    # percentile scores in pctl_levels_all (min 10th, max 90th)
    MOD_FRAC = 0.05
    
    if ADJUST_MEANS:
        # depending on whether dist is skewed left or right, will need to 
        # adjust lower (<5th) or upper (>95th) tail of dist
        mean_med_ratio = ASHE_dict['mean_salary'][i]/ASHE_dict['median_salary'][i]
        if (not np.isnan(mean_med_ratio)):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore", category=RuntimeWarning)
                curr_mean = np.mean(curr_synth_dist)
            gamma, N = ASHE_dict['mean_salary'][i]/curr_mean, len(curr_synth_dist)

            if mean_med_ratio>1:
                slice_start_ix = int((1-MOD_FRAC)*N)
                #print(slice_start_ix)
                sliced_arr = np.copy(curr_synth_dist[slice_start_ix::])
                curr_synth_dist[slice_start_ix::] = sliced_arr*(
                    gamma*N*curr_mean - np.sum(curr_synth_dist[0:slice_start_ix]) ) /(
                    np.sum(sliced_arr)
                    )
            elif mean_med_ratio<1:
                slice_stop_ix = int(MOD_FRAC*N)
                sliced_arr = np.copy(curr_synth_dist[0:slice_stop_ix])
                curr_synth_dist[0:slice_stop_ix] = sliced_arr*(
                    gamma*N*curr_mean - np.sum(curr_synth_dist[slice_stop_ix::]) ) /(
                    np.sum(sliced_arr)
                    )
            
        # --- diagnostics ---
        # print(S2023['mean_salary'][i], np.mean(curr_synth_dist)) # <-- should be equal
        
    # --- diagnostics ---
    #print(len(curr_synth_dist)) # should all be length 9*N_samples or 0
    
    # add samples from the current synthetic dist to overall one if SOC2020 code
    # is associated with an MS occupation in 2023
    if code in soc_codes:
        if len(curr_synth_dist)>0:
            ms_salary_synth.extend(np.random.choice(curr_synth_dist, int(
                ms_jobs[soc_codes==code].sum())))

### Means and medians from percentile-matching salary distributions

In [14]:
# Mathematical sciences

MS_med = int(np.median(ms_salary_synth)*hr_to_annual)
MS_mean = int(np.mean(ms_salary_synth)*hr_to_annual)

# UK wide estimates: mean and median

UK_med = ASHE_dict['median_salary'][0]*hr_to_annual
UK_mean = ASHE_dict['mean_salary'][0]*hr_to_annual

print('=== 2023 salaries ===\n')

print(f'MS median: £{MS_med:,.0f}; UK median: £{UK_med:,.0f}')
print(f'MS premium (median): {100*(MS_med/UK_med-1):.0f}%')

print(f'\nMS mean: £{MS_mean:,.0f}; UK mean: £{UK_mean:,.0f}')
print(f'MS premium (mean): {100*(MS_mean/UK_mean-1):.0f}%')

=== 2023 salaries ===

MS median: £45,785; UK median: £34,963
MS premium (median): 31%

MS mean: £52,081; UK mean: £42,210
MS premium (mean): 23%


### Convert notebook to HTML

In [15]:
%%bash 
jupyter nbconvert --to html 02_salary_analysis.ipynb

[NbConvertApp] Converting notebook 02_salary_analysis.ipynb to html
[NbConvertApp] Writing 598369 bytes to 02_salary_analysis.html
